In [ ]:
import numpy as np
import os
import cv2
import scipy.io
from scipy.spatial import KDTree
from scipy.ndimage import gaussian_filter

def generate_density_map(image_shape, points, k=11, beta=0.15):

    H, W = image_shape
    density_map = np.zeros((H, W), dtype=np.float32)
    
    if len(points) == 0:
        return density_map

    tree = KDTree(points)

    for i, point in enumerate(points):
        distances, locations = tree.query(point, k=k+1)
        if len(distances) > 1:
            avg_dist = np.mean(distances[1:]) 
        else:
            avg_dist = np.mean(distances)

        sigma = beta * avg_dist

        x, y = int(point[0]), int(point[1])

        size = int(6 * sigma)
        if size % 2 == 0:
            size += 1
        gaussian_map = np.zeros((size, size), dtype=np.float32)
        center = size // 2
        gaussian_map[center, center] = 1

        gaussian_map = gaussian_filter(gaussian_map, sigma, mode='constant')

        x1 = max(0, x - center)
        y1 = max(0, y - center)
        x2 = min(W, x + center + 1)
        y2 = min(H, y + center + 1)

        gx1 = center - (x - x1)
        gy1 = center - (y - y1)
        gx2 = gx1 + (x2 - x1)
        gy2 = gy1 + (y2 - y1)

        g_patch = gaussian_map[gy1:gy2, gx1:gx2]
        d_patch = density_map[y1:y2, x1:x2]

        h = min(g_patch.shape[0], d_patch.shape[0])
        w = min(g_patch.shape[1], d_patch.shape[1])

        if h > 0 and w > 0:   
            density_map[y1:y1+h, x1:x1+w] += gaussian_map[gy1:gy1+h, gx1:gx1+w]

    return density_map


In [6]:
# Combined function to process data
def process_dataset(image_dir, gt_dir, npy_output_dir, img_output_dir, sigma):
    os.makedirs(npy_output_dir, exist_ok=True)
    os.makedirs(img_output_dir, exist_ok=True)
    image_files = os.listdir(image_dir)

    for image_file in image_files:
        image_path = os.path.join(image_dir, image_file)
        mat_path = os.path.join(gt_dir, "GT_" + image_file.replace(".jpg", ".mat"))

        image = cv2.imread(image_path)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        H, W, _ = image.shape

        mat = scipy.io.loadmat(mat_path)
        points = mat["image_info"][0, 0][0, 0][0]
        density_map = generate_density_map((H, W), points, sigma)

        npy_path = os.path.join(npy_output_dir, image_file.replace(".jpg", ".npy"))
        np.save(npy_path, density_map)

        normed_map = (density_map / np.max(density_map) * 255).astype(np.uint8)
        colored_map = cv2.applyColorMap(normed_map, cv2.COLORMAP_JET)
        img_path = os.path.join(img_output_dir, image_file.replace('.jpg', '.png'))
        cv2.imwrite(img_path, colored_map)

In [ ]:
sigmas = [6,7,8,9,10,11,12,13,14,15]
image_dir='crowd_wala_dataset/train_data/images/'
gt_dir='crowd_wala_dataset/train_data/ground_truth/'

for sigma in sigmas:
    npy_output_dir=f'density_maps/train_data_sigma_{sigma}_new2/'
    img_output_dir=f'density_maps_images/train_data_sigma_{sigma}_new2/'
    process_dataset(image_dir, gt_dir, npy_output_dir, img_output_dir, sigma)
    print(f"processed sigma = {sigma}")

# k=9, beta=0.15 => sigma 9 -> mse: 4.159608840942383, mse: 22.065937042236328, Variance: 4.763588905334473

processed sigma = 9
